<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
%reset -f
from pythonrc import *

ModuleNotFoundError: No module named 'pyquaternion'

# Simple test

## Setup the environment and the configuration

Load the allocator with Minimum Magnitude reference filter

In [ ]:
allocator = skadipy.allocator.PseudoInverse(
    actuators=[
        tunnel,
        voithschneider_port,
        voithschneider_starboard
    ],
    force_torque_components=[
        skadipy.allocator.ForceTorqueComponent.X,
        skadipy.allocator.ForceTorqueComponent.Y,
        skadipy.allocator.ForceTorqueComponent.N
    ]
)

allocator.compute_configuration_matrix()

In [ ]:
data = generate_spiral_dataset(
    num_points=1000,
    num_turns=5,
    k=1
)

We need to send vectors $\tau = [X, Y, Z, K, M, N]^\top$. Even if the number of
controlled DOFs are different, we still need to send a vector with the same size.

Say, we control $\tau_{mode=1} = [X, Y, N]^\top$, than we should simply send
```python
>>> X, Y, Z = 0, 0, 0
>>> tau = np.array([X, Y, 0, 0, 0, N])
```

In [ ]:
history_f = []

for i in range(n := data.shape[0]):
    X = data[i, 0]
    Y = data[i, 1]
    tau = np.array([[X, Y, 0, 0, 0, 0]], dtype=np.float32).T
    xi = allocator.allocate(tau)

    history_f.append(f := allocator.allocated)

F = np.concatenate([i.T for i in history_f])


In [ ]:
plt.scatter(data[:, 0], data[:, 1], s=4)
plt.scatter(F[:, 0], F[:, 1], s=1)
plt.title("Input forces")
plt.xlabel("X")
plt.ylabel("Y")